In [8]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import re, os, csv, pathlib
import pandas as pd
from statistics import mean, variance
accents = ['african', 'indian', 'hongkong', 'philippines', 
           'england', 'scotland', 'ireland', 'australia', 
           'canada', 'us', 
           'bermuda', 'southatlandtic', 'wales', 'malaysia']
accent_short_forms = {'hongkong':'hk', "african":"afr", "philippines":"phil", "indian":"ind",  
                      "england":"eng", 'scotland':'sco', 'ireland':'ire',
                      "us":"us", "canada":"can", "australia":"aus",
                      'bermuda':'ber', 'southatlandtic':'satl', 'wales':'wal', 'malaysia':'mal'
                     }

def replace_with_short_forms(s):
    for key, value in accent_short_forms.items():
        s = s.replace(key, value)
    return s

def last_name(pth):
    return pathlib.PurePath(pth).name

def get_dirs(pth):
    return [last_name(f.name) for f in os.scandir(pth) if f.is_dir()]

def get_each_run(lne):
    return list(map(float, re.findall(': (.+?) -> ', lne)[0].split(' ')))

def get_selection_counts(s):
    return list(map(replace_with_short_forms, re.findall('Counter\\((.+?)\\)', s)))

def get_test_file_from_stats_path(run_number, stats_file_opened):
    return stats_file_opened.name[:-9]+"run_{}/output/test_infer_log.txt".format(run_number)

def WER_test_file(test_file):
    txt_file = open(test_file, 'r')
    lines = txt_file.readlines()
    matched = ""
    for line in lines:
        if "==========>>>>>>Evaluation Greedy WER: " in line:
            txt_file.close()
            return float(line.rstrip().split(": ")[1])
    txt_file.close()
    return ""

def get_eta(func, eta):
    if func != "FL1MI":
        return ""
    else:
        return "-n:"+str(float(eta[4:]))

In [12]:
budget = 500
target = 20
features = 'w2v2'

In [13]:
# sample_path = 'Error-Driven-ASR-Personalization/CMU_expts/accent/hindi/manifests/TSS_output/all/budget_100/target_50/FL1MI/eta_1.0/euclidean/39/stats.txt'
cols = ['accent', 'ground', 'function', 'feature', 'duration', 'samples', 
        'WER-r1', 'WER-r2', 'WER-r3', 'WER-mean', 'WER-var', 'accents-run_1', 'accents-run_2', 'accents-run_3']
df = pd.DataFrame(columns = cols)
# accents = ['canada', 'england', 'australia', 'us', 'indian', 'philippines', 'african']
accents = ['african', 'indian', 'hongkong', 'philippines', 
           'england', 'scotland', 'ireland', 'australia', 
           'canada', 'us', 
           'bermuda', 'southatlandtic', 'wales', 'malaysia']
cnt = 0
pretrain_added = dict([(accent, False) for accent in accents])
for accent in accents:
    if not(pathlib.Path('./{}/manifests/TSS_output/'.format(accent)).is_dir()):
        continue
    pretrain_log = '{}/manifests/TSS_output/test_infer_log.txt'.format(accent)
    pick_from = 'without'
    if not(pathlib.Path('./{}/manifests/TSS_output/{}/budget_{}/target_{}/'.format(accent, pick_from, budget, target)).is_dir()):
        continue
    for function in get_dirs('./{}/manifests/TSS_output/{}/budget_{}/target_{}/'.format(accent, pick_from, budget, target)):
        if function == "random":
            stats_file_path = './{}/manifests/TSS_output/{}/budget_{}/target_{}/{}/stats.txt'.format(accent, pick_from, budget, 
                                                                                                                target, function)                                                                                  
            if not(os.path.isfile(stats_file_path)):
                continue
            stats_file = open(stats_file_path, 'r')
            lines = stats_file.readlines()
            total_selections, total_durations, accented_selections, accented_durations = map(get_each_run, lines[:4])
            sample_frac = mean([x[0]/x[1] for x in zip(accented_selections, total_selections)])
            sample_total = mean(total_selections)
            duration_frac = mean([x[0]/x[1] for x in zip(accented_durations, total_durations)])
            duration_total = mean(total_durations)
            df_duration = "{:.2f}/{:.2f}".format(duration_total*duration_frac, duration_total)
            df_samples = "{:.2f}/{:.2f}".format(sample_total*sample_frac, sample_total)
            df_selections = get_selection_counts(lines[5])
            try:
                wers = [WER_test_file(get_test_file_from_stats_path(i, stats_file)) for i in range(1,4)]
                df_wer_mean = round(mean(wers), 2)
                df_wer_var = round(variance(wers), 3)
            except:
                continue
                print("no WER's in file", get_test_file_from_stats_path(1, stats_file))
                wers = [0,0,0]
                df_wer_mean = 0
                df_wer_var = 999
            df = df.append(dict(zip(cols, [accent, pick_from, function, "NA", df_duration, df_samples]+wers
                                    +[df_wer_mean, df_wer_var] + df_selections)), ignore_index=True)
            stats_file.close()
            continue
#         for eta in get_dirs('./{}/manifests/TSS_output/{}/budget_{}/target_{}/{}/'.format(accent, pick_from, budget, target, function)):
#             for similarity in get_dirs('./{}/manifests/TSS_output/{}/budget_{}/target_{}/{}/{}/'.format(accent, pick_from, 
#                                                                                                     budget, target, function, eta)):
#                 cnt += 1
#                 stats_file_path = './{}/manifests/TSS_output/{}/budget_{}/target_{}/{}/{}/{}/{}/stats.txt'.format(accent, pick_from, budget, 
#                                                                                                             target, function, eta, similarity, 
#                                                                                                             features)                                                                                  
#                 if not(os.path.isfile(stats_file_path)):
#                     continue
#                 stats_file = open(stats_file_path, 'r')
#                 lines = stats_file.readlines()
#                 total_selections, total_durations, accented_selections, accented_durations = map(get_each_run, lines[:4])
#                 sample_frac = mean([x[0]/x[1] for x in zip(accented_selections, total_selections)])
#                 sample_total = mean(total_selections)
#                 duration_frac = mean([x[0]/x[1] for x in zip(accented_durations, total_durations)])
#                 duration_total = mean(total_durations)
#                 df_duration = "{:.2f}/{:.2f}".format(duration_total*duration_frac, duration_total)
#                 df_samples = "{:.2f}/{:.2f}".format(sample_total*sample_frac, sample_total)
#                 df_selections = get_selection_counts(lines[5])
#                 try:
#                     wers = [WER_test_file(get_test_file_from_stats_path(i, stats_file)) for i in range(1,4)]
#                     df_wer_mean = round(mean(wers), 2)
#                     df_wer_var = round(variance(wers), 3)
#                 except:
#                     continue
#                     print("no WER's in file", get_test_file_from_stats_path(1, stats_file))
#                     wers = [0,0,0]
#                     df_wer_mean = 0
#                     df_wer_var = 999
#                 df = df.append(dict(zip(cols, [accent, pick_from, function+get_eta(function, eta), features, df_duration, df_samples]+wers
#                                         +[df_wer_mean, df_wer_var] + df_selections)), ignore_index=True)
#                 if not(pretrain_added[accent]) and os.path.isfile(pretrain_log):
#                     pretrain_added[accent] = True
#                     pre_WER = WER_test_file(pretrain_log)
#                     df = df.append(dict(zip(cols, [accent, "pretrain", '', '', '', '']+['-']*3+[pre_WER, ''])), ignore_index=True)
#                 stats_file.close()

In [14]:
df = df.sort_values(by=['accent', 'feature', 'ground', 'function'], ascending=True, ignore_index=True)
display(df)

,accent,ground,function,feature,duration,samples,WER-r1,WER-r2,WER-r3,WER-mean,WER-var,accents-run_1,accents-run_2,accents-run_3


In [5]:
# total selection : 100 100 100 -> 100.00
# total selection duration: 357.0149433106577 357.0149433106577 357.0149433106577 -> 357.01
# accented selection: 76 76 76 -> 76.00
# accented duration: 254.74947845804974 254.74947845804974 254.74947845804974 -> 254.75

# all selections: [Counter({'hindi': 76, 'korean': 8, 'spanish': 7, 'arabic': 3, 'chinese': 3, 'vietnamese': 3}), Counter({'hindi': 76, 'korean': 8, 'spanish': 7, 'arabic': 3, 'chinese': 3, 'vietnamese': 3}), Counter({'hindi': 76, 'korean': 8, 'spanish': 7, 'arabic': 3, 'chinese': 3, 'vietnamese': 3})]

#Evaluation Greedy WER: 16.19

df.to_csv("without_{}_{}_{}.csv".format(budget, target, features), index=False)